In [1]:
#!/usr/bin/env python
# coding: utf-8
# In[1]:
#!/usr/bin/env python
# coding: utf-8
# get_ipython().run_line_magic('matplotlib', 'inline')
import pandas as pd
import time
import numpy as np
from datetime import datetime
from sklearn.externals import joblib 
import os
import glob
from konlpy.tag import Mecab
import lightgbm as lgb
print(lgb.__version__)
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib 
from sklearn.model_selection import StratifiedKFold, KFold
import gc
from tqdm import tqdm_notebook, tqdm
import json
from typing import NamedTuple
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

import warnings
warnings.filterwarnings(action='ignore')
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR
print(torch.__version__)
# from tools import eval_summary, save_feature_importance, merge_preds
from tools import EarlyStopping

device = torch.device('cpu')
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

/home/aiden/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


2.3.2
1.3.1
GeForce RTX 2070 SUPER


device(type='cuda', index=0)

In [2]:
# In[2]:
torch.set_num_threads(8)
torch.get_num_threads()

8

#### Load Data

In [3]:
df_train = pd.read_csv('input/train.csv', dtype=np.float32)
df_test = pd.read_csv('input/test.csv', dtype=np.float32)
print(df_train.shape, df_test.shape)


(810000, 230) (10000, 227)


In [4]:
layer_cols = [c for c in df_train.columns if 'layer_' in c]
fea_cols = [c for c in df_train.columns if c not in layer_cols]

len(fea_cols), len(layer_cols)

(226, 4)

In [5]:
df_model = df_train

#### Model

##### DNN1Model

In [6]:
# df1 = pd.read_csv('input/SiO2.txt', sep='\t')
# df2 = pd.read_csv('input/Si3N4.txt', sep='\t')

# df_nk = pd.merge(df1, df2, on='Wavelength(nm)')
# df_nk = df_nk[:226]

# df_nk.columns

In [7]:
class DNN1Model(torch.nn.Module):
    def __init__(self, input_size, dropout_probability=0.3):
        super().__init__()
        relu = torch.nn.ReLU()
        dropout = torch.nn.Dropout(p=dropout_probability)

        self.layer_1 = torch.nn.Sequential(
            torch.nn.Linear(input_size, input_size), relu, torch.nn.BatchNorm1d(input_size), dropout, 
            torch.nn.Linear(input_size, input_size), relu, torch.nn.BatchNorm1d(input_size), dropout, 
            torch.nn.Linear(input_size, input_size), relu, torch.nn.BatchNorm1d(input_size), dropout, 
        )
        self.layer_2 = torch.nn.Sequential(
            torch.nn.Linear(input_size, input_size), relu, torch.nn.BatchNorm1d(input_size), dropout, 
            torch.nn.Linear(input_size, input_size), relu, torch.nn.BatchNorm1d(input_size), dropout, 
            torch.nn.Linear(input_size, input_size), relu, torch.nn.BatchNorm1d(input_size), dropout, 
        )
        self.layer_3 = torch.nn.Sequential(
            torch.nn.Linear(input_size, input_size), relu, torch.nn.BatchNorm1d(input_size), dropout, 
            torch.nn.Linear(input_size, input_size), relu, torch.nn.BatchNorm1d(input_size), dropout, 
            torch.nn.Linear(input_size, input_size), relu, torch.nn.BatchNorm1d(input_size), dropout, 
        )
        self.layer_4 = torch.nn.Sequential(
            torch.nn.Linear(input_size, input_size), relu, torch.nn.BatchNorm1d(input_size), dropout, 
            torch.nn.Linear(input_size, input_size), relu, torch.nn.BatchNorm1d(input_size), dropout, 
            torch.nn.Linear(input_size, input_size), relu, torch.nn.BatchNorm1d(input_size), dropout, 
        )
        
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(input_size, input_size), relu,
            torch.nn.Linear(input_size, input_size), relu,
            torch.nn.Linear(input_size, input_size), relu,
            torch.nn.Linear(input_size, 4),
        )
        
        self.layer13_n = torch.Tensor(df_nk[['n_x']].T.values).to(device)
        self.layer13_k = torch.Tensor(df_nk[['k_x']].T.values).to(device)
        self.layer24_n = torch.Tensor(df_nk[['n_y']].T.values).to(device)
        self.layer24_k = torch.Tensor(df_nk[['k_y']].T.values).to(device)
        
    
    def forward(self, x_fea):
        
        out_layer_1 = self.layer_1(torch.add(torch.mul(x_fea, self.layer13_n), self.layer13_k))
        out_layer_2 = self.layer_2(torch.add(torch.mul(out_layer_1, self.layer24_n), self.layer24_k))
        out_layer_3 = self.layer_3(torch.add(torch.mul(out_layer_2, self.layer13_n), self.layer13_k))
        out_layer_4 = self.layer_4(torch.add(torch.mul(out_layer_2, self.layer24_n), self.layer24_k))
        
        return self.fc(out_layer_4)
    

##### CNNModel

In [8]:
# # size = 48
# W = 15 # input_volume_size
# F = 6  # kernel_size
# S = 1   # strides
# P = 1
# # padding_size

# size = (W - F + 2*P) / S + 1
# size
# # ((size - 1) * S) - 2*P + F

In [9]:
class CNNModel(torch.nn.Module):
    def __init__(self, dropout_probability=0.3):
        super().__init__()
        relu = torch.nn.ReLU()
        dropout = torch.nn.Dropout(p=dropout_probability)

        self.cnn = torch.nn.Sequential(
            torch.nn.Conv1d(1, 2, 31, stride=1, padding=0), #196
            relu, torch.nn.MaxPool1d(2), #98 
            torch.nn.Conv1d(2, 4, 19, stride=1, padding=0), #80
            relu, torch.nn.MaxPool1d(2), #40
            torch.nn.Conv1d(4, 8, 11, stride=1, padding=0), #30
            relu, torch.nn.MaxPool1d(2), #15
            torch.nn.Conv1d(8, 16, 6, stride=1, padding=1), #12
            relu, torch.nn.MaxPool1d(2), #6
        )
            
# #             torch.nn.Linear(input_size, 4),
#             torch.nn.Linear(input_size, 200), relu, #torch.nn.BatchNorm1d(200), dropout, 
#             torch.nn.Linear(200, 200), relu, #torch.nn.BatchNorm1d(200), dropout,
#             torch.nn.Linear(200, 200), relu, #torch.nn.BatchNorm1d(200), dropout,
#             torch.nn.Linear(200, 150), relu, #torch.nn.BatchNorm1d(200), dropout,
#             torch.nn.Linear(150, 128), relu, #torch.nn.BatchNorm1d(128), dropout,
#             torch.nn.Linear(128, 128), relu, #torch.nn.BatchNorm1d(128), dropout,
#             torch.nn.Linear(128, 100), relu, #torch.nn.BatchNorm1d(128), dropout,
#             torch.nn.Linear(100, 64), relu, #torch.nn.BatchNorm1d(128), dropout,
#             torch.nn.Linear(64, 32), relu, #torch.nn.BatchNorm1d(128), dropout,
#             torch.nn.Linear(32, 16), relu, #torch.nn.BatchNorm1d(128), dropout,
#             torch.nn.Linear(16, 8), relu, #torch.nn.BatchNorm1d(128), dropout,
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(16*6, 64), relu, #torch.nn.BatchNorm1d(128), dropout,
            torch.nn.Linear(64, 4)
       )
    def forward(self, x):
        x = x.unsqueeze(1)
        out = self.cnn(x)
        dim = 1
        for d in out.size()[1:]: #24, 4, 4
            dim = dim * d
        out = out.view(-1, dim)
        out = self.fc(out)
        return out

        return self.model(x)
    

##### DNNModel

In [10]:
class DNNModel(torch.nn.Module):
    def __init__(self, input_size, dropout_probability=0.3):
        super(DNNModel,self).__init__()
#         relu = torch.nn.ReLU()
        act = torch.nn.ELU()
        dropout = torch.nn.Dropout(p=dropout_probability)

        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size, 2048), torch.nn.BatchNorm1d(2048), act, dropout, 
            torch.nn.Linear(2048, 2048), torch.nn.BatchNorm1d(2048), act, dropout, 
            torch.nn.Linear(2048, 1024), torch.nn.BatchNorm1d(1024), act, dropout,
            torch.nn.Linear(1024, 1024), torch.nn.BatchNorm1d(1024), act, dropout,            
            torch.nn.Linear(1024, 512), torch.nn.BatchNorm1d(512), act, dropout,
            torch.nn.Linear(512, 512), torch.nn.BatchNorm1d(512), act, dropout,
            torch.nn.Linear(512, 256), torch.nn.BatchNorm1d(256), act, dropout,            
            torch.nn.Linear(256, 4)
        )
    def forward(self, x):
        return self.model(x)
 

#### DataSet

In [11]:
   
class SemiDataset(Dataset):
    def __init__(self, df, fea_cols, y_cols):        
        self.X = df[fea_cols].values
        self.y = df[y_cols].values
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx].astype(np.float32), self.y[idx].astype(np.float32)
    

#### Trainer

In [12]:
class Trainer(object):
    def __init__(self, model, criterion, optimizer, scheduler, device):
        self.device = device
        self.model = model#.to(self.device)
        self.criterion = criterion#.to(self.device)
        self.optimizer = optimizer
        self.scheduler = scheduler

        print(self.model.train())
        pass
    
    def train(self, data_loader):
        self.model.train()
        total_loss = 0
        for data in data_loader:
            X_batch, y_batch = data
            X_batch = X_batch.to(self.device)
            y_batch = y_batch.to(self.device)
            
            y_pred = self.model(X_batch)
#             print(y_pred, y_batch)
            
            loss = self.criterion(y_pred, y_batch)
            total_loss = total_loss + loss.item()

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            self.scheduler.step()
        
        return total_loss / len(data_loader)
    
    def eval(self, data_loader):
        self.model.eval()
        total_loss = 0
#         print('valid_loader', len(valid_loader))
        for data in data_loader:
            X_batch, y_batch = data
            X_batch = X_batch.to(self.device)
            y_batch = y_batch.to(self.device)
            with torch.no_grad():
                y_pred = self.model(X_batch)
                loss = self.criterion(y_pred, y_batch)
                total_loss = total_loss + loss.item()
        return total_loss / len(data_loader)

    def save(self, model_path='checkpoint.pt'):
        torch.save(self.model.state_dict(), 'checkpoint.pt')
        return
    
    def load(self, model_path='checkpoint.pt'):
        self.model.load_state_dict(torch.load(model_path))
        return

#### Train

In [13]:
model_ts = datetime.now().strftime('%Y%m%dT%H%M%S')
print(model_ts)

print(f'fea_size {len(fea_cols)} layer_cols {layer_cols}')

20200122T201112
fea_size 226 layer_cols ['layer_1', 'layer_2', 'layer_3', 'layer_4']


In [14]:
torch.manual_seed(81511991154)
torch.initial_seed()

81511991154

In [15]:
dataset = SemiDataset(df_model[fea_cols + layer_cols], fea_cols, layer_cols)

train_set, val_set = torch.utils.data.random_split(dataset, [700000, 110000])

print(len(train_set), len(val_set))

batch_size = 50000
num_workers = 4

loader_params = {    
    'batch_size' : batch_size,
    'shuffle' : True,
    'num_workers' : num_workers,
    'drop_last' : False,
}
train_loader = DataLoader(dataset=train_set, **loader_params)
val_loader = DataLoader(dataset=val_set, **loader_params)

print(f'batch_size {batch_size} num_workers {num_workers}')
print(f'train_loader {len(train_loader)} val_loader {len(val_loader)}')

700000 110000
batch_size 50000 num_workers 4
train_loader 14 val_loader 3


In [16]:
# model = DNN1Model(input_size=len(fea_cols), dropout_probability=0.3).to(device)
model = DNNModel(input_size=len(fea_cols), dropout_probability=0.1).to(device)
#     model = CNNModel(dropout_probability=0.1).to(device)
# model = joblib.load('checkpoint.model').to(device)

    
criterion = nn.L1Loss(reduction='mean').to(device)
# criterion = nn.MSELoss(reduction='mean').to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)
# optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

scheduler = StepLR(optimizer, step_size=300, gamma=0.9)


trainer = Trainer(model, criterion, optimizer, scheduler, device)
# trainer.load()

DNNModel(
  (model): Sequential(
    (0): Linear(in_features=226, out_features=2048, bias=True)
    (1): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ELU(alpha=1.0)
    (3): Dropout(p=0.1, inplace=False)
    (4): Linear(in_features=2048, out_features=2048, bias=True)
    (5): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ELU(alpha=1.0)
    (7): Dropout(p=0.1, inplace=False)
    (8): Linear(in_features=2048, out_features=1024, bias=True)
    (9): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ELU(alpha=1.0)
    (11): Dropout(p=0.1, inplace=False)
    (12): Linear(in_features=1024, out_features=1024, bias=True)
    (13): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ELU(alpha=1.0)
    (15): Dropout(p=0.1, inplace=False)
    (16): Linear(in_features=1024, out_features=512, bias=True)
    (17): BatchNorm1d(512,

In [17]:
early_stopping = EarlyStopping(patience=50, min_epoch=100, verbose=True)

total_epoch = 2000

for e in tqdm_notebook(range(total_epoch), total=total_epoch, desc='Epoch'):
    if os.path.isfile('stop.flag'):
        print(f'{e} stop!')
        break

    train_loss = trainer.train(train_loader)
    valid_loss = trainer.eval(val_loader)
    
    ts = datetime.now().strftime('%Y%m%dT%H%M%S')
    print(f'[{ts}] Epock {e} / {total_epoch} train_loss: {train_loss}  valid_loss: {valid_loss}')
    
    early_stopping(valid_loss, model)

    if early_stopping.early_stop:
        print("\tEarly stopping epoch {}, valid loss {}".format(e, valid_loss))
        
    if e % 100 == 0:
        for param_group in optimizer.param_groups:
            print("Current learning rate is: {}".format(param_group['lr']))

model.load_state_dict(torch.load('checkpoint.pt'))
torch.save(model.state_dict(), 'model/{}_{}.model'.format(model_ts, early_stopping.best_epoch))
    
# torch.save(model.state_dict(), f'checkpoint.pt.{train_loss}')

[20200122T201122] Epock 0 / 2000 train_loss: 150.56396702357702  valid_loss: 87.64200337727864
Validation loss decreased (inf --> 87.64200338).  Saving model ...
Current learning rate is: 0.01
[20200122T201130] Epock 1 / 2000 train_loss: 133.2375204903739  valid_loss: 142.87035115559897
EarlyStopping 2 / 100 counter: 1 out of 50
[20200122T201138] Epock 2 / 2000 train_loss: 104.80827331542969  valid_loss: 103.3559799194336
EarlyStopping 3 / 100 counter: 2 out of 50
[20200122T201146] Epock 3 / 2000 train_loss: 74.40599332536969  valid_loss: 71.6258036295573
Validation loss decreased (87.64200338 --> 71.62580363).  Saving model ...
[20200122T201154] Epock 4 / 2000 train_loss: 63.18517630440848  valid_loss: 73.10904184977214
EarlyStopping 5 / 100 counter: 1 out of 50
[20200122T201202] Epock 5 / 2000 train_loss: 61.304196221487864  valid_loss: 64.25983174641927
Validation loss decreased (71.62580363 --> 64.25983175).  Saving model ...
[20200122T201210] Epock 6 / 2000 train_loss: 59.26798657

Traceback (most recent call last):
  File "/home/aiden/anaconda3/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/aiden/anaconda3/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/aiden/anaconda3/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/aiden/anaconda3/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/home/aiden/anaconda3/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/aiden/anaconda3/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/aiden/anaconda3/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(he

KeyboardInterrupt: 

In [ ]:
# model = joblib.load('checkpoint.model')
model.eval()
y_pred = model(torch.Tensor(df_test[fea_cols].values).to(device))    
print(y_pred)

In [ ]:
df_submit = pd.read_csv('input/sample_submission.csv', index_col=0)
df_submit[layer_cols] = y_pred.cpu().detach().numpy()
ts = datetime.now().strftime('%Y%m%dT%H%M%S')
df_submit.to_csv(f'submit/{ts}_{valid_loss}.csv')
print(ts, valid_loss)